In [1]:
#! /usr/bin/env python

from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local[2]"))

In [2]:
import re

def parse_article(line):
    try:
        article_id, text = line.rstrip().split('\t', 1)
    except ValueError as e:
        return []
    text = re.sub(r"^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split(r"\W*\s+\W*", text, flags=re.UNICODE)
    return words

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article).cache()

In [3]:
from operator import add

FIRST_WORD="narodnaya"

def find_pairs(article):
    memorize = None
    result = []
    for word in article:
        word = word.strip().lower()
        if memorize:
            result.append(("%s_%s" % (memorize, word), 1))
            memorize = None
        if word == FIRST_WORD:
            memorize = word
    return result

pairs = wiki.flatMap(find_pairs).reduceByKey(add).sortByKey()

In [4]:
for pair in pairs.collect():
    print("%s\t%s" % (pair[0], pair[1]))

narodnaya_gazeta	1
narodnaya_volya	9
